전주 4/15 미팅 피드백 내용
- cnn 모델에 들어가는 feature, feature 사이즈 얼마인지(어떻게 일정하게 잘랐는지)  
- transformer 이해하고, 우리만의 모델을 어떤식으로 보완할 지 <미흡>
    

----

## tag train에서 들어가는 `input` 확인하기 
논문 아키텍쳐에서 logmel이 cnn10()의 feature 로 들어가게 되고 output 은 tag (multi hot vector) 이다.(multi classfication)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

from audio_tag.data_loader import tag_loader #training data 불러오기
import numpy as np
from hparams import hparams as hp
from encoder import Tag
from tqdm import tqdm #상태바를 나타내는 패키지

In [33]:
#하이퍼 파라미터
class_num = 300 #최대 빈도수 300개 단어를 targeting
device = torch.device("cuda")#gpu 지정
data_dir = hp.data_dir
learning_rate=1e-3
model = Tag(class_num).to(device)#Tag 모델 결합

In [32]:
#data_dir 지정 하기
from pathlib import Path
# data(default)
data_dir = Path(r'./create_dataset/data/data_splits')
eval_data_dir = r'./create_dataset/data/data_splits/evaluation'
train_data_dir = r'./create_dataset/data/data_splits/development'
test_data_dir = r'./create_dataset/data/test_data'
word_dict_pickle_path = r'./create_dataset/data/pickles/words_list.p'
word_freq_pickle_path = r'./create_dataset/data/pickles/words_frequencies.p'


In [34]:

from typing import Tuple, List, AnyStr, Union
from pathlib import Path

from numpy import ndarray, recarray
from torch.utils.data import Dataset
from numpy import load as np_load
import pickle
import torch
import numpy as np
import os
from torch import cat as pt_cat, zeros as pt_zeros, \
    ones as pt_ones, from_numpy, Tensor
from torch.utils.data.dataloader import DataLoader


class TagDataset(Dataset):
    #development,evaluation

    def __init__(self, data_dir: Path,
                 split: AnyStr,class_num):
        super(TagDataset, self).__init__()
        the_dir: Path = data_dir.joinpath(split)

        self.examples: List[Path] = sorted(the_dir.iterdir())[::5]
        with open('./audio_tag/audioTagNum_{}_fin.pickle'.format(split), 'rb') as f:
            self.tags = pickle.load(f)

    def __len__(self) \
            -> int:
        return len(self.examples)

    def __getitem__(self,
                    item: int) \
            -> Tuple[ndarray, ndarray]:
        ex: Union[Path, recarray] = self.examples[item]
        ex: recarray = np_load(str(ex), allow_pickle=True)
        features = ex['features'].item()
        file_name = ex['file_name'].item()
        tag = self.tags[file_name]
        return features, tag

def collate_fn(batch,input_pad_at = 'start'):

    in_t_steps = max([i[0].shape[0] for i in batch])

    in_dim = batch[0][0].shape[-1]

    input_tensor, output_tensor = [], []

    for in_b, out_b in batch:
        if in_t_steps >= in_b.shape[0]:
            padding = pt_zeros(in_t_steps - in_b.shape[0], in_dim).float()
            data = [from_numpy(in_b).float()]
            if input_pad_at.lower() == 'start':
                data.insert(0, padding)
            else:
                data.append(padding)
            tmp_in: Tensor = pt_cat(data)
        else:
            tmp_in: Tensor = from_numpy(in_b[:in_t_steps, :]).float()
        input_tensor.append(tmp_in.unsqueeze_(0))
        tmp_out: Tensor = torch.Tensor(out_b)
        output_tensor.append(tmp_out.unsqueeze_(0))


    input_tensor = pt_cat(input_tensor)
    output_tensor = pt_cat(output_tensor)

    return input_tensor, output_tensor

def tag_loader(data_dir,split,batch_size,class_num=300,shuffle=True,drop_last=True):
    dataset = TagDataset(data_dir,split,class_num)
    return DataLoader(
        dataset=dataset, batch_size=batch_size,
        shuffle=shuffle, drop_last=drop_last, collate_fn=collate_fn)



In [90]:
dataset[0][0]

array([[-19.847643 , -15.194129 , -12.097026 , ..., -44.278336 ,
        -44.402115 , -43.560978 ],
       [-15.420743 , -12.454311 , -15.403213 , ..., -42.60773  ,
        -43.07943  , -43.69768  ],
       [ -7.7899404,  -8.5197115, -10.9066105, ..., -42.75554  ,
        -43.462414 , -43.95652  ],
       ...,
       [-13.9696865, -11.698841 , -13.525093 , ..., -42.922016 ,
        -42.006817 , -42.786354 ],
       [-11.543285 , -10.471132 , -14.285076 , ..., -42.262226 ,
        -42.98683  , -43.665268 ],
       [-11.735426 , -13.872566 , -15.717896 , ..., -44.456345 ,
        -45.00557  , -42.84395  ]], dtype=float32)

In [89]:
len(dataset)

2893

In [91]:
dataset[0][0].shape

(2231, 64)

tagdata는 길이가 각각 다른것을 확인할 수 있다. 길이를 고정 시키기 위해 
위 `tag_loader`함수의 `collate_fn` 함수를 사용자 정의하여 불러온다

In [80]:
#길이가 다른 input을 batch로 만들기 위해 배치를 만들어주는 collate_fn조작
#batch=2 이면 input=[sample1, sample2]

def collate_fn(batch,input_pad_at = 'start'):

    in_t_steps = max([i[0].shape[0] for i in batch])

    in_dim = batch[0][0].shape[-1]

    input_tensor, output_tensor = [], []

    for in_b, out_b in batch:
        if in_t_steps >= in_b.shape[0]:
            padding = pt_zeros(in_t_steps - in_b.shape[0], in_dim).float()
            data = [from_numpy(in_b).float()]
            if input_pad_at.lower() == 'start':
                data.insert(0, padding)
            else:
                data.append(padding)
            tmp_in: Tensor = pt_cat(data)
        else:
            tmp_in: Tensor = from_numpy(in_b[:in_t_steps, :]).float()
        input_tensor.append(tmp_in.unsqueeze_(0))
        tmp_out: Tensor = torch.Tensor(out_b)
        output_tensor.append(tmp_out.unsqueeze_(0))


    input_tensor = pt_cat(input_tensor)
    output_tensor = pt_cat(output_tensor)

    return input_tensor, output_tensor

dataloader 참조 https://sanghyu.tistory.com/90

tag data(developmet)에 쓰이는 데이터 파악하기

In [37]:
import pickle
with open('./audio_tag/audioTagNum_development_fin.pickle', 'rb') as f:
    tags = pickle.load(f)

In [39]:
import pandas as pd
df_tags=pd.DataFrame(tags)

In [40]:
df_tags

,Distorted AM Radio noise.wav,Paper_Parchment_Rustling.wav,03 Whales Slowing Down.wav,Rope tied to boat in port.wav,carpenter bee.wav,violin1.wav,121006_rain&amp;storm.wav,draaiorgel2.wav,Officejet Printer.wav,Stockholm subway C20 train departing.wav,...,Sink faucet.wav,sapjjr-train-grandcentral-i6.wav,"Car Passes, Countryside.wav",Fast Flowing Stream.wav,Mild rain.wav,"Crickets calling at night, UK.wav","Rain On Town Square, South Germany.wav",amazon 04.wav,steps come and go on stone ground in a village with bird ambient.wav,Heavy rain - some people 2.wav
0,0,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,1,0,1,1,0
1,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,1,0,1,0,0,1
296,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
297,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
298,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [42]:


#tag 모델에 들어가는 traing_data 
training_data = tag_loader(data_dir=data_dir, split='development',
                                      batch_size=16,class_num=class_num)

In [77]:
from typing import Tuple, List, AnyStr, Union
from pathlib import Path

from numpy import ndarray, recarray
from torch.utils.data import Dataset
from numpy import load as np_load
import pickle
import torch
import numpy as np
import os
from torch import cat as pt_cat, zeros as pt_zeros, \
    ones as pt_ones, from_numpy, Tensor
from torch.utils.data.dataloader import DataLoader

class TagDataset(Dataset):
    #development,evaluation

    def __init__(self, data_dir: Path,
                 split: AnyStr,class_num):
        super(TagDataset, self).__init__()
        the_dir: Path = data_dir.joinpath(split)

        self.examples: List[Path] = sorted(the_dir.iterdir())[::5]
        with open('./audio_tag/audioTagNum_{}_fin.pickle'.format(split), 'rb') as f:
            self.tags = pickle.load(f)

    def __len__(self) \
            -> int:
        return len(self.examples)

    def __getitem__(self,
                    item: int) \
            -> Tuple[ndarray, ndarray]:
        ex: Union[Path, recarray] = self.examples[item]
        ex: recarray = np_load(str(ex), allow_pickle=True)
        features = ex['features'].item()
        file_name = ex['file_name'].item()
        tag = self.tags[file_name]
        return features, tag


In [48]:
#훈련 데이터 셋 저장
split='development'
dataset=TagDataset(data_dir,split,class_num)

In [50]:
the_dir = data_dir.joinpath(split)

TypeError: 'PosixPath' object is not subscriptable

In [73]:
examples=sorted(the_dir.iterdir())[::5]

[PosixPath('create_dataset/data/data_splits/development/clotho_file_(Door) Porte entree.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_, iannaeneee iiaca, caienu niaoe iaao aaaiiaie, nooe eie n, oi e naeno aaoa ia neiinoe, caii eaaaaoia iiiineon.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_00294 harvest festival rumour 1.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_00332 lake beach 1.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_00708 christmas background 1.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_01 car sequence.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_01 residential elevator with door open and close.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_01-31 Footsteps, Wood, Barefoot, Jumps &amp; Scuffs.wav_0.npy'),
 PosixPath('create_datas

In [68]:
sorted(the_dir.iterdir())[::5]#1개만 사용

[PosixPath('create_dataset/data/data_splits/development/clotho_file_(Door) Porte entree.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_, iannaeneee iiaca, caienu niaoe iaao aaaiiaie, nooe eie n, oi e naeno aaoa ia neiinoe, caii eaaaaoia iiiineon.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_00294 harvest festival rumour 1.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_00332 lake beach 1.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_00708 christmas background 1.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_01 car sequence.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_01 residential elevator with door open and close.wav_0.npy'),
 PosixPath('create_dataset/data/data_splits/development/clotho_file_01-31 Footsteps, Wood, Barefoot, Jumps &amp; Scuffs.wav_0.npy'),
 PosixPath('create_datas

In [8]:
class_num = 300
device = torch.device("cuda")
data_dir = hp.data_dir
learning_rate=1e-3
model = Tag(class_num).to(device)
training_data = tag_loader(data_dir=data_dir, split='development',
                                      batch_size=16,class_num=class_num)
test_data = tag_loader(data_dir=data_dir, split='evaluation',
                               batch_size=16,class_num=class_num)
optimizer =torch.optim.Adam(model.parameters(), lr=learning_rate,
        betas=(0.9, 0.999), eps=1e-08, weight_decay=0., amsgrad=True)
# optimizer = torch.optim.Adam(model.parameters(), lr=hp.lr, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.98)
tag_loss = nn.BCELoss()

def train(epoch):
    # bar  = tqdm(training_data,total=len(training_data))
    loss_list = []
    model.train()
    with tqdm(training_data,total=len(training_data)) as bar:
        for i, (feature, tag) in enumerate(bar):
            feature = feature.to(device)
            tag = tag.to(device)
            optimizer.zero_grad()
            out_tag = model(feature)
            loss = tag_loss(out_tag,tag)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
            bar.set_description("epoch:{} idx:{} loss:{:.6f}".format(epoch, i, np.mean(loss_list)))
    return np.mean(loss_list)

def test(epoch):
    eva_loss = []
    model.eval()
    with torch.no_grad():
        for i, (feature, tag) in enumerate(test_data):
            feature = feature.to(device)
            tag = tag.to(device)
            out_tag = model(feature)
            loss = tag_loss(out_tag, tag)
            eva_loss.append(loss.item())
    mean_loss = np.mean(eva_loss)
    print("epoch:{:d}--testloss:{:.6f}".format(epoch,mean_loss.item()))

    # return  mean_loss


if __name__ == '__main__':
    train_b = True
    epoch_last = 0
    if train_b:
        # model.load_state_dict(torch.load("./models/280/TagModel_{}.pt".format(str(40))))
        for epoch in range(epoch_last+1,10):
            train(epoch)
            scheduler.step(epoch)
            test(epoch)
            if epoch%5==0:
                torch.save(model.state_dict(),'./models/tag_models/TagModel_{}.pt'.format(epoch))
    else:
        for epoch in range(0,225,5):
            model.load_state_dict(torch.load("./models/tag_models/TagModel_{}.pt".format(str(epoch))))
            test(epoch)

    



  0%|          | 0/180 [00:00<?, ?it/s]

epoch:1--testloss:0.165132


  0%|          | 0/180 [00:00<?, ?it/s]

epoch:2--testloss:0.160427


  0%|          | 0/180 [00:00<?, ?it/s]

...

epoch:7--testloss:0.155342


  0%|          | 0/180 [00:00<?, ?it/s]

epoch:8--testloss:0.151327


epoch:9 idx:179 loss:0.150429: 100%|██████████| 180/180 [00:47<00:00,  3.77it/s]


epoch:9--testloss:0.149380


In [10]:
class_num = 300
device = torch.device("cuda")
data_dir = hp.data_dir
learning_rate=1e-3
model = Tag(class_num).to(device)
training_data = tag_loader(data_dir=data_dir, split='development',
                                      batch_size=16,class_num=class_num)
test_data = tag_loader(data_dir=data_dir, split='evaluation',
                               batch_size=16,class_num=class_num)
optimizer =torch.optim.Adam(model.parameters(), lr=learning_rate,
        betas=(0.9, 0.999), eps=1e-08, weight_decay=0., amsgrad=True)
# optimizer = torch.optim.Adam(model.parameters(), lr=hp.lr, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.98)
tag_loss = nn.BCELoss()

In [22]:
features

NameError: name 'features' is not defined

In [11]:
print(training_data)

In [34]:
    for i, (feature, tag) in enumerate(bar):
        print("feature:",feature)
        print("tag:",tag)

feature: tensor([[[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         ...,
         [ -14.6726,  -10.9286,   -6.5536,  ...,  -34.9652,  -36.5822,
           -46.7000],
         [ -11.4504,   -9.2350,   -8.1468,  ...,  -34.6178,  -35.7291,
           -46.2654],
         [ -12.0941,  -10.4113,   -9.9724,  ...,  -36.7973,  -38.7754,
           -48.8406]],

        [[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         ...,
         [  -9.1071,   -9.0306,   -9.9549,  ...,  -31.1102,  -31.3776,
           -32.2908],
         [  -9.5249,   -7.1261,

In [28]:
len(training_data)

180

In [9]:
training_data

In [37]:
feature[0].shape

torch.Size([2580, 64])

In [38]:
feature[1].shape

torch.Size([2580, 64])

In [39]:
feature[2].shape

torch.Size([2580, 64])

In [40]:
feature.shape # 16= batch size, 

torch.Size([16, 2580, 64])

In [ ]:
#batch_size=16, 512, T/16, mel_bins/16  T/16=2580  41280

In [44]:
tag[0].shape

torch.Size([300])

In [45]:
tag[1].shape

torch.Size([300])

In [46]:
len(feature)

16

In [29]:
len(test_data)

65

In [3]:
'''https://github.com/qiuqiangkong/audioset_tagging_cnn/blob/master/pytorch/models.py'''

import torch
from torch import nn
import torch.nn.functional as F
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):

        super(ConvBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1),
                               padding=(1, 1), bias=False)

        self.conv2 = nn.Conv2d(in_channels=out_channels,
                               out_channels=out_channels,
                               kernel_size=(3, 3), stride=(1, 1),
                               padding=(1, 1), bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):

        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x


class Cnn10(nn.Module):
    def __init__(self):

        super(Cnn10, self).__init__()

        self.bn0 = nn.BatchNorm2d(64)

        self.conv_block1 = ConvBlock(in_channels=1, out_channels=64)
        self.conv_block2 = ConvBlock(in_channels=64, out_channels=128)
        self.conv_block3 = ConvBlock(in_channels=128, out_channels=256)
        self.conv_block4 = ConvBlock(in_channels=256, out_channels=512)
        self.init_weight()

    def init_weight(self):
        init_bn(self.bn0)

    def forward(self, input):

        x = input.unsqueeze(1)   # (batch_size, 1, time_steps, mel_bins)

        x = x.transpose(1, 3)
        x = self.bn0(x)
        x = x.transpose(1, 3)

        x = self.conv_block1(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block2(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block3(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv_block4(x, pool_size=(2, 2), pool_type='avg')
        x = F.dropout(x, p=0.2, training=self.training)     #(batch_size, 512, T/16, mel_bins/16)

        return x

class Tag(nn.Module):
    def __init__(self,class_num):
        super(Tag, self).__init__()
        self.feature = Cnn10()
        self.fc1 = nn.Linear(512,512,bias=True)
        self.fc = nn.Linear(512,class_num,bias=True)
        self.init_weights()

    def init_weights(self):
        init_layer(self.fc1)
        init_layer(self.fc)

    def forward(self,input):
        '''
        :param input: (batch_size,time_steps, mel_bins)
        :return: ()
        '''
        x = self.feature(input)     #(batch_size, 512, T/16, mel_bins/16)
        x = torch.mean(x,dim=3)     #(batch_size, 512, T/16)
        (x1, _) = torch.max(x, dim=2)
        x2 = torch.mean(x, dim=2)
        x = x1 + x2
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu_(self.fc1(x))
        #(batch_size,class_num)
        output = torch.sigmoid(self.fc(x))
        # output = self.fc(x)

        return output



In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [13]:
summary(model,(512,64)) #tag-model summary

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1           [-1, 64, 512, 1]             128
            Conv2d-2          [-1, 64, 512, 64]             576
       BatchNorm2d-3          [-1, 64, 512, 64]             128
            Conv2d-4          [-1, 64, 512, 64]          36,864
       BatchNorm2d-5          [-1, 64, 512, 64]             128
         ConvBlock-6          [-1, 64, 256, 32]               0
            Conv2d-7         [-1, 128, 256, 32]          73,728
       BatchNorm2d-8         [-1, 128, 256, 32]             256
            Conv2d-9         [-1, 128, 256, 32]         147,456
      BatchNorm2d-10         [-1, 128, 256, 32]             256
        ConvBlock-11         [-1, 128, 128, 16]               0
           Conv2d-12         [-1, 256, 128, 16]         294,912
      BatchNorm2d-13         [-1, 256, 128, 16]             512
           Conv2d-14         [-1, 256, 

In [21]:
summary(model,(512,64),batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1           [16, 64, 512, 1]             128
            Conv2d-2          [16, 64, 512, 64]             576
       BatchNorm2d-3          [16, 64, 512, 64]             128
            Conv2d-4          [16, 64, 512, 64]          36,864
       BatchNorm2d-5          [16, 64, 512, 64]             128
         ConvBlock-6          [16, 64, 256, 32]               0
            Conv2d-7         [16, 128, 256, 32]          73,728
       BatchNorm2d-8         [16, 128, 256, 32]             256
            Conv2d-9         [16, 128, 256, 32]         147,456
      BatchNorm2d-10         [16, 128, 256, 32]             256
        ConvBlock-11         [16, 128, 128, 16]               0
           Conv2d-12         [16, 256, 128, 16]         294,912
      BatchNorm2d-13         [16, 256, 128, 16]             512
           Conv2d-14         [16, 256, 

In [26]:
summary(model,(2580,64),batch_size=16)#t_step = 2580

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1          [16, 64, 2580, 1]             128
            Conv2d-2         [16, 64, 2580, 64]             576
       BatchNorm2d-3         [16, 64, 2580, 64]             128
            Conv2d-4         [16, 64, 2580, 64]          36,864
       BatchNorm2d-5         [16, 64, 2580, 64]             128
         ConvBlock-6         [16, 64, 1290, 32]               0
            Conv2d-7        [16, 128, 1290, 32]          73,728
       BatchNorm2d-8        [16, 128, 1290, 32]             256
            Conv2d-9        [16, 128, 1290, 32]         147,456
      BatchNorm2d-10        [16, 128, 1290, 32]             256
        ConvBlock-11         [16, 128, 645, 16]               0
           Conv2d-12         [16, 256, 645, 16]         294,912
      BatchNorm2d-13         [16, 256, 645, 16]             512
           Conv2d-14         [16, 256, 

In [3]:
!nvidia-smi

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for
       usage information.

Mon Apr 19 19:42:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0  On |                  N/A |
| 35%   37C   

In [2]:
!kill -9 26731

NameError: name 'model' is not defined

In [8]:
import torch
import torch.nn as nn
import time

from data_handling import get_clotho_loader, get_test_data_loader
from model import TransformerModel  # , RNNModel, RNNModelSmall
import itertools
import numpy as np
import os
import sys
import logging
import csv

from util import get_file_list, get_padding, print_hparams, greedy_decode, \
    calculate_bleu, calculate_spider, LabelSmoothingLoss, beam_search, align_word_embedding, gen_str
from hparams import hparams
from torch.utils.tensorboard import SummaryWriter

import argparse

hp = hparams()
parser = #argparse.ArgumentParser(description='hparams for model')

device = torch.device(hp.device)
np.random.seed(hp.seed)
torch.manual_seed(hp.seed)


def train():
    model.train()
    total_loss_text = 0.
    start_time = time.time()
    batch = 0
    for src, tgt, tgt_len, ref in training_data:
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_pad_mask = get_padding(tgt, tgt_len)
        tgt_in = tgt[:, :-1]
        tgt_pad_mask = tgt_pad_mask[:, :-1]
        tgt_y = tgt[:, 1:]

        optimizer.zero_grad()
        output = model(src, tgt_in, target_padding_mask=tgt_pad_mask)

        loss_text = criterion(output.contiguous().view(-1, hp.ntoken), tgt_y.transpose(0, 1).contiguous().view(-1))
        loss = loss_text
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), hp.clip_grad)
        optimizer.step()
        total_loss_text += loss_text.item()

        writer.add_scalar('Loss/train-text', loss_text.item(), (epoch - 1) * len(training_data) + batch)

        batch += 1

        if batch % hp.log_interval == 0 and batch > 0:
            mean_text_loss = total_loss_text / hp.log_interval
            elapsed = time.time() - start_time
            current_lr = [param_group['lr'] for param_group in optimizer.param_groups][0]
            logging.info('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2e} | ms/batch {:5.2f} | '
                         'loss-text {:5.4f}'.format(
                epoch, batch, len(training_data), current_lr,
                elapsed * 1000 / hp.log_interval, mean_text_loss))
            total_loss_text = 0
            start_time = time.time()